In [1]:
import pandas as pd, numpy as np
from pathlib import Path
import zarr
import pandas as pd
from collections import Counter

In [2]:
#with out background
full_without_bg = "/hpc/group/jilab/rz179/cellpt/combined/semi_balanced/combined_meta_full_downsampled_no_bonecells.csv"

Subset_20pct_without_bg = "/hpc/group/jilab/rz179/cellpt/combined/semi_balanced/combined_meta_full_downsampled_no_bonecells_subset20pct.csv"
Subset_20pct_without_bg_train_csv = "/hpc/group/jilab/rz179/cellpt/combined/semi_balanced/20pct_train.csv"
Subset_20pct_without_bg_val_csv   = "/hpc/group/jilab/rz179/cellpt/combined/semi_balanced/20pct_val.csv"
Subset_20pct_without_bg_test_csv  = "/hpc/group/jilab/rz179/cellpt/combined/semi_balanced/20pct_test.csv"


#with backgound
full_bg = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv"

Subset_20pct_bg = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse_20pct.csv"
Subset_20pct_bg_train_csv = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_train.csv"
Subset_20pct_bg_val_csv   = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_val.csv"
Subset_20pct_bg_test_csv  = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/20pct_test.csv"


#with background human only
full_human_only_bg = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only_20to1.csv"

train_human_only_bg = "/hpc/group/jilab/rz179/subset_per_slide_two_eval_splits_full/train.csv"

In [3]:
import pandas as pd
import csv

# Method 1: Process in chunks
def get_slide_ids_chunked(filepath, chunksize=10000):
    unique_slide_ids = set()
    
    print(f"Processing file in chunks of {chunksize}...")
    
    try:
        chunk_count = 0
        for chunk in pd.read_csv(filepath, chunksize=chunksize):
            chunk_count += 1
            #print(f"Processing chunk {chunk_count}...")
            
            # Add unique slide_ids from this chunk
            unique_slide_ids.update(chunk['slide_id'].unique())
            
            # Print progress
            #print(f"  Total unique slide_ids so far: {len(unique_slide_ids)}")
            
    except Exception as e:
        print(f"Error: {e}")
        return None
        
    return sorted(list(unique_slide_ids))

# Method 2: Using basic CSV reader (even more memory efficient)
def get_slide_ids_csv_reader(filepath):
    unique_slide_ids = set()
    
    try:
        with open(filepath, 'r') as file:
            reader = csv.DictReader(file)
            
            row_count = 0
            for row in reader:
                row_count += 1
                unique_slide_ids.add(row['slide_id'])
                
                # Print progress every 100k rows
                if row_count % 100000 == 0:
                    print(f"Processed {row_count:,} rows, found {len(unique_slide_ids)} unique slide_ids")
                    
    except Exception as e:
        print(f"Error: {e}")
        return None
        
    return sorted(list(unique_slide_ids))

# File path
#full_bg = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_downsampled_no_bonecells_withCoarse.csv"
full_human_only_bg = "/hpc/group/jilab/rz179/cellpt/combined/withBackground/combined_meta_human_only_20to1.csv"

# Try Method 1 first (pandas with chunks)
print("Method 1: Using pandas with chunking...")
slide_ids = get_slide_ids_chunked(full_human_only_bg, chunksize=5000)

# Display results
if slide_ids is not None:
    print(f"\nFound {len(slide_ids)} unique slide IDs:")
    for slide_id in slide_ids:
        print(slide_id)
    
    # Save to file
    with open('unique_slide_ids_humanonly.txt', 'w') as f:
        for slide_id in slide_ids:
            f.write(f"{slide_id}\n")
    
    print(f"\nSlide IDs saved to 'unique_slide_ids_humanonly.txt'")
else:
    print("Both methods failed. The file might be corrupted or have format issues.")




Method 1: Using pandas with chunking...
Processing file in chunks of 5000...

Found 28 unique slide IDs:
Xenium_Preview_Human_Lung_Cancer
Xenium_Preview_Human_Non_diseased_Lung
Xenium_Prime_Ovarian_Cancer_FFPE
Xenium_V1_FFPE_Human_Brain_Alzheimers
Xenium_V1_FFPE_Human_Brain_Glioblastoma
Xenium_V1_FFPE_Human_Brain_Healthy
Xenium_V1_hBoneMarrow_acute_lymphoid
Xenium_V1_hBoneMarrow_nondiseased
Xenium_V1_hBone_nondiseased
Xenium_V1_hColon_Cancer_Add_on
Xenium_V1_hColon_Cancer_Base
Xenium_V1_hColon_Non_diseased_Add_on
Xenium_V1_hColon_Non_diseased_Base
Xenium_V1_hHeart_nondiseased
Xenium_V1_hKidney_cancer
Xenium_V1_hKidney_nondiseased
Xenium_V1_hLiver_cancer
Xenium_V1_hLiver_nondiseased
Xenium_V1_hLung_cancer
Xenium_V1_hLymphNode_nondiseased
Xenium_V1_hPancreas_Cancer_Add_on
Xenium_V1_hPancreas_nondiseased
Xenium_V1_hSkin_nondiseased_section_1
Xenium_V1_hSkin_nondiseased_section_2
Xenium_V1_hTonsil_reactive_follicular_hyperplasia
Xenium_human_Lung_Cancer_FFPE
Xenium_human_Lymph_Node_FFPE
Xe

In [7]:
file_path = Subset_20pct_bg
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Myeloid cells: 20,000 (4.90%)
Smooth muscle cells: 20,000 (4.90%)
Fibroblasts: 20,000 (4.90%)
Epithelial cells: 20,000 (4.90%)
Endothelial cells: 20,000 (4.90%)
T cells: 20,000 (4.90%)
B cells: 20,000 (4.90%)
Cardiac muscle cells: 20,000 (4.90%)
Stromal cells: 20,000 (4.90%)
Erythrocytes: 20,000 (4.90%)
Neurons: 20,000 (4.90%)
Ovary cancer cells: 20,000 (4.90%)
Stem and progenitor cells: 20,000 (4.90%)
Colon cancer cells: 20,000 (4.90%)
Oligodendrocytes: 19,103 (4.68%)
NK cells: 18,580 (4.56%)
Skin cancer cells: 14,605 (3.58%)
Mesangial cells: 13,615 (3.34%)
Lung cancer cells: 12,175 (2.99%)
Astrocytes: 11,228 (2.75%)
Pericytes: 10,894 (2.67%)
Pancreas cancer cells: 10,121 (2.48%)
Skeletal muscle cells: 6,131 (1.50%)
Schwann cells: 4,944 (1.21%)
Microglia: 4,384 (1.07%)
Liver cancer cells: 2,084 (0.51%)


In [16]:
file_path = full_human_only_bg
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Fine class Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")
    
print("Total samples:")
print(total)

Fine class Counts and distribution:
Ovary cancer cells: 100,000 (9.11%)
Colon cancer cells: 100,000 (9.11%)
T cells: 89,816 (8.19%)
NK cells: 89,552 (8.16%)
B cells: 82,600 (7.53%)
Stem and progenitor cells: 77,147 (7.03%)
Fibroblasts: 75,404 (6.87%)
Myeloid cells: 73,277 (6.68%)
Epithelial cells: 66,469 (6.06%)
Endothelial cells: 61,953 (5.65%)
Lung cancer cells: 60,874 (5.55%)
Smooth muscle cells: 56,475 (5.15%)
Pancreas cancer cells: 50,607 (4.61%)
Oligodendrocytes: 30,822 (2.81%)
Pericytes: 27,497 (2.51%)
Stromal cells: 24,079 (2.19%)
Liver cancer cells: 10,418 (0.95%)
Microglia: 7,628 (0.70%)
Astrocytes: 7,432 (0.68%)
Neurons: 5,134 (0.47%)
Total samples:
1097184


In [13]:
file_path = full_human_only_bg
label_col = "cell_type_coarse"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Coarse class Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Coarse class Counts and distribution:
Immune cells: 335,245 (30.56%)
Cancer cells: 321,899 (29.34%)
Stromal cells: 126,980 (11.57%)
Stem and progenitor cells: 77,147 (7.03%)
Epithelial cells: 66,469 (6.06%)
Endothelial cells: 61,953 (5.65%)
Muscle cells: 56,475 (5.15%)
Glial cells: 45,882 (4.18%)
Neurons: 5,134 (0.47%)


In [10]:
file_path = full_bg
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Myeloid cells: 100,000 (4.90%)
Smooth muscle cells: 100,000 (4.90%)
Fibroblasts: 100,000 (4.90%)
Epithelial cells: 100,000 (4.90%)
Endothelial cells: 100,000 (4.90%)
T cells: 100,000 (4.90%)
B cells: 100,000 (4.90%)
Cardiac muscle cells: 100,000 (4.90%)
Stromal cells: 100,000 (4.90%)
Erythrocytes: 100,000 (4.90%)
Neurons: 100,000 (4.90%)
Ovary cancer cells: 100,000 (4.90%)
Stem and progenitor cells: 100,000 (4.90%)
Colon cancer cells: 100,000 (4.90%)
Oligodendrocytes: 95,514 (4.68%)
NK cells: 92,899 (4.56%)
Skin cancer cells: 73,025 (3.58%)
Mesangial cells: 68,076 (3.34%)
Lung cancer cells: 60,874 (2.99%)
Astrocytes: 56,141 (2.75%)
Pericytes: 54,472 (2.67%)
Pancreas cancer cells: 50,607 (2.48%)
Skeletal muscle cells: 30,656 (1.50%)
Schwann cells: 24,718 (1.21%)
Microglia: 21,921 (1.07%)
Liver cancer cells: 10,418 (0.51%)


In [3]:

file_path = train_human_only_bg
label_col = "cell_type"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Fibroblasts: 19,601 (8.00%)
Smooth muscle cells: 19,598 (8.00%)
Oligodendrocytes: 16,032 (6.54%)
Colon cancer cells: 15,183 (6.20%)
Ovary cancer cells: 15,183 (6.20%)
Epithelial cells: 14,193 (5.79%)
Endothelial cells: 14,192 (5.79%)
Stem and progenitor cells: 14,192 (5.79%)
T cells: 14,166 (5.78%)
NK cells: 14,142 (5.77%)
B cells: 13,493 (5.51%)
Myeloid cells: 12,604 (5.14%)
Pericytes: 11,920 (4.87%)
Lung cancer cells: 11,429 (4.66%)
Stromal cells: 10,796 (4.41%)
Pancreas cancer cells: 10,339 (4.22%)
Microglia: 5,340 (2.18%)
Astrocytes: 5,202 (2.12%)
Liver cancer cells: 3,799 (1.55%)
Neurons: 3,594 (1.47%)


In [4]:
file_path = train_human_only_bg
label_col = "cell_type_coarse"

counts = Counter()
total = 0

# Stream the CSV in chunks
for chunk in pd.read_csv(file_path, usecols=[label_col], chunksize=100_000):
    vals = chunk[label_col]
    counts.update(vals)
    total += len(vals)

# Print results
print("Counts and distribution:")
for label, count in counts.most_common():
    pct = count / total * 100
    print(f"{label}: {count:,} ({pct:.2f}%)")

Counts and distribution:
Cancer cells: 55,933 (22.83%)
Immune cells: 54,405 (22.21%)
Stromal cells: 42,317 (17.27%)
Glial cells: 26,574 (10.85%)
Muscle cells: 19,598 (8.00%)
Epithelial cells: 14,193 (5.79%)
Endothelial cells: 14,192 (5.79%)
Stem and progenitor cells: 14,192 (5.79%)
Neurons: 3,594 (1.47%)
